###  Ybigta 19기 신입기수프로젝트 5조 와인추천시스템 중 Tokenizing부분입니다

In [1]:
pip install konlpy

     |████████████████████████████████| 19.4 MB 95.3 MB/s 
     |████████████████████████████████| 448 kB 41.3 MB/s 
     |████████████████████████████████| 86 kB 4.0 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
import pandas as pd
import numpy as np
import konlpy
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.tokenize import word_tokenize
from konlpy.tag import Okt,Kkma
okt = Okt()
kkma = Kkma()
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
wine = pd.read_csv("/content/drive/My Drive/Colab Notebooks/total_.csv")  

In [5]:
wine.head()

,Unnamed: 0,name_ko,name_en,price,type,country,city,aroma,food,rate_expert,rate_consumer,alcohol,sugar,acidity,body,tannin,maker's_note,tasting_note,posting_title,posting_text,wine_title,wine_text,hashtag,blog_text
0,0,더 롱 리틀 독 스파클링,The Long Little Dog Sparkling,30000.0,스파클링,프랑스,랑그독 루씨용,"복숭아, 살구, 꽃, 흰꽃, 레몬","리코타, 파스타, 샐러드",-,3.5,NaN,1.0,5.0,2.0,1.0,"밝고 맑은 레몬 컬러에 레몬, 살구, 복숭아, 흰꽃의 아로마가 느껴진다. 입 안에서...",NaN,NaN,NaN,NaN,NaN,NaN,WINE\n더 롱 리틀 독 스파클링 [ The Long Little Dog Spar...
1,1,루베롱 블랑,LUBERON BLANC,25000.0,NaN,프랑스,론,"헤이즐넛, 꽃, 아카시아, 레몬",NaN,-,4,NaN,1.0,3.0,3.0,1.0,"화사한 노란색을 띠며 레몬, 아카시아 꽃, 헤이즐넛 향이 어우러진다. 입 안에서는 ...",NaN,NaN,NaN,NaN,NaN,NaN,와인이야기\n[하남 포도내음 와인아울렛/와인샵]와인소개_화이트와인추천_Marreno...
2,2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼","Marie Stuart, Cuvee de la Reine Brut",145000.0,스파클링,프랑스,샹파뉴,꽃,"아페리티프, 파스타",-,4,12 %,1.0,4.0,3.0,1.0,"반짝이는 연한 금빛의 정교한 거품을 지녔으며, 흰 꽃 패션 프루츠, 달콤한 흰 과일...",NaN,와인21닷컴이 선정한 7월 이달의 와인,조금 늦게 찾아온 장마와 무더위가 기다리고 있는 7월. 장마철의 습한 기운을 떨쳐내...,여왕의 샴페인,1867년 랭스(Reims)에 설립된 마리 스튜아트는 스코틀랜드의 여왕이자 프랑스 ...,NaN,BEST와인추천\n와인21닷컴이 선정한 7월 이달의 와인\n와인21 ・ 2021. ...
3,3,"마스카 델 타코, 로시 피노 네로 로사토","Masca del Tacco, “ Ro’Si ” Pinot Nero Rosato",38000.0,NaN,이탈리아,뿔리아,"딸기, 미네랄",NaN,-,4,12~13 %,1.0,3.0,2.0,1.0,밟은 옅은 분홍색에 야생 딸기 젤리 수박과 같은 달콤한 향이 풍부하고 그 뒤로 향신...,NaN,NaN,NaN,NaN,NaN,NaN,이탈리아와인\nMASCA DEL TACCO [마스카 델 타코] 제이픽와인\n산타 ・...
4,4,"비에티, 바르베라 다스티 '트레 비그네'","Vietti, Barbera d'Asti 'Tre Vigne'",50000.0,NaN,이탈리아,아스티,"체리, 오크",NaN,-,4,NaN,1.0,3.0,3.0,4.0,바르베라 다스티의 포도는 라 크레나 포도밭으로 유명한 아스티 언덕주변인 아그리아노 ...,NaN,NaN,NaN,NaN,NaN,NaN,"(N) Piemonte\nVietti, Barbera d'Asti Tre Vigne..."


In [6]:
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 806 entries, 0 to 805
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     806 non-null    int64  
 1   name_ko        805 non-null    object 
 2   name_en        805 non-null    object 
 3   price          304 non-null    float64
 4   type           68 non-null     object 
 5   country        325 non-null    object 
 6   city           309 non-null    object 
 7   aroma          321 non-null    object 
 8   food           214 non-null    object 
 9   rate_expert    325 non-null    object 
 10  rate_consumer  325 non-null    object 
 11  alcohol        241 non-null    object 
 12  sugar          325 non-null    float64
 13  acidity        325 non-null    float64
 14  body           325 non-null    float64
 15  tannin         325 non-null    float64
 16  maker's_note   317 non-null    object 
 17  tasting_note   28 non-null     object 
 18  posting_ti

###### 먼저, 수치형 데이터 보다는 블로그나 기사 등을 통해 특징을 추출하는 것이 목표이기 때문에 해당 열들을 지웠습니다.

###### posting_text의 경우, 불필요한 말이 너무 많아 오히려 분석의 성능을 저하시킨다고 판단해 지웠습니다

In [7]:
del wine["Unnamed: 0"]
del wine["name_en"]
del wine["price"]
del wine["country"]
del wine["city"]
del wine["rate_expert"]
del wine["rate_consumer"]
del wine["sugar"]
del wine["alcohol"]
del wine["acidity"]
del wine["tannin"]
del wine["posting_text"]
del wine["food"]
del wine["body"]

In [8]:
wine.head()

,name_ko,type,aroma,maker's_note,tasting_note,posting_title,wine_title,wine_text,hashtag,blog_text
0,더 롱 리틀 독 스파클링,스파클링,"복숭아, 살구, 꽃, 흰꽃, 레몬","밝고 맑은 레몬 컬러에 레몬, 살구, 복숭아, 흰꽃의 아로마가 느껴진다. 입 안에서...",NaN,NaN,NaN,NaN,NaN,WINE\n더 롱 리틀 독 스파클링 [ The Long Little Dog Spar...
1,루베롱 블랑,NaN,"헤이즐넛, 꽃, 아카시아, 레몬","화사한 노란색을 띠며 레몬, 아카시아 꽃, 헤이즐넛 향이 어우러진다. 입 안에서는 ...",NaN,NaN,NaN,NaN,NaN,와인이야기\n[하남 포도내음 와인아울렛/와인샵]와인소개_화이트와인추천_Marreno...
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼",스파클링,꽃,"반짝이는 연한 금빛의 정교한 거품을 지녔으며, 흰 꽃 패션 프루츠, 달콤한 흰 과일...",NaN,와인21닷컴이 선정한 7월 이달의 와인,여왕의 샴페인,1867년 랭스(Reims)에 설립된 마리 스튜아트는 스코틀랜드의 여왕이자 프랑스 ...,NaN,BEST와인추천\n와인21닷컴이 선정한 7월 이달의 와인\n와인21 ・ 2021. ...
3,"마스카 델 타코, 로시 피노 네로 로사토",NaN,"딸기, 미네랄",밟은 옅은 분홍색에 야생 딸기 젤리 수박과 같은 달콤한 향이 풍부하고 그 뒤로 향신...,NaN,NaN,NaN,NaN,NaN,이탈리아와인\nMASCA DEL TACCO [마스카 델 타코] 제이픽와인\n산타 ・...
4,"비에티, 바르베라 다스티 '트레 비그네'",NaN,"체리, 오크",바르베라 다스티의 포도는 라 크레나 포도밭으로 유명한 아스티 언덕주변인 아그리아노 ...,NaN,NaN,NaN,NaN,NaN,"(N) Piemonte\nVietti, Barbera d'Asti Tre Vigne..."


###### 분석의 성능을 위해 반복되는 불필요한 어구들을 정규표현식을 사용해 지웠습니다. (기자이름, 기사를 쓴 날, 평점 등)

In [9]:
import re

In [10]:
for i  in range(0, 806):
    wine.tasting_note[i] = re.sub("\n.{1,}\n[0-9]{4}[.][0-9]{2}[.][0-9]{2}\n[A-Z0-9]{2,}\s빈티지\n[0-9][.][0-9]"," ", str(wine.tasting_note[i]))

In [11]:
wine.posting_title.unique()

array([nan, '와인21닷컴이 선정한 7월 이달의 와인', '와인21추천 BEST OF BEST, 선물하기 좋은 와인',
       '올해 밸런타인데이에 함께할 와인!', '졸업 시즌의 축하주, 스파클링 와인',
       '와인21추천 BEST OF BEST, 꼭 한번 마셔볼만한 마트 와인', '초여름밤, 설레는 기분을 선사해줄 와인',
       "와인21추천 BEST OF BEST, 집에서 더 즐겁게 '홈파티 와인'", '늦여름에 마시기 좋은 레드 와인',
       '와인21추천 BEST OF BEST - 치명적 우아함, 부르고뉴(Bourgogne)',
       '[와인추천] 소소한 야외 파티에서 즐기기 좋은 와인', '와인21닷컴이 선정한 3월 이달의 와인',
       '부부의 날을 기념하며 마시기 좋은 와인', '와인21닷컴이 선정한 2월 이달의 와인',
       '호캉스에 단 한 병의 와인을 가지고 간다면?', '와인21닷컴이 선정한 5월 이달의 와인',
       '와인21닷컴이 선정한 6월 이달의 와인', '와인21닷컴이 선정한 8월 이달의 와인',
       '와인21닷컴이 선정한 9월 이달의 와인', '와인21닷컴이 선정한 10월 이달의 와인',
       '와인21닷컴이 선정한 11월 이달의 와인', '와인21닷컴이 선정한 12월 이달의 와인',
       '와인21닷컴이 선정한 2021년 1월 이달의 와인', '와인21닷컴이 선정한 4월 이달의 와인',
       '폭염이 지속되는 여름밤에 마시고 싶은 와인', '무더운 날 칠링해서 마시기 좋은 레드 와인은?',
       '삼계탕과 잘 어울리는 와인은?', '봄바람과 함께, 야외에서 즐기는 와인',
       '행사가 많은 5월, 선물이 고민인가요?', '춘곤증을 날려주는 청량감 넘치는 와인',
       '미세먼지가 심한 날, 홈술 추천 와인', '벚꽃 필 무렵엔 이 와인을!', '봄날, 지금 썸타는 중이라면?',
       "와인21's PI

In [12]:
for i  in range(0, 806):
    wine.posting_title[i] = re.sub("와인21추천"," ", str(wine.posting_title[i]))
    wine.posting_title[i] = re.sub("이달의 와인"," ", str(wine.posting_title[i]))
    wine.posting_title[i] = re.sub("BEST OF BEST"," ", str(wine.posting_title[i]))    
    wine.posting_title[i] = re.sub("와인"," ", str(wine.posting_title[i]))    
    wine.posting_title[i] = re.sub("21닷컴이 선정한"," ", str(wine.posting_title[i]))        

###### 불필요하다고 판단된 어구들이 지워진 이후 unique 값들을 확인해보았습니다

In [13]:
wine.tasting_note.unique()

array(['nan', ' \n세인트 넘 조아요~~~~\n\n굿뜨베리...', '  \n샤도네이를 좋아하게 해준 와인',
       ' \n양고기 허브솔트와 마리아주',
       ' \n오픈직후 피어오르는 잘익은 과실향이 인상적이나 한모금하는순간 할퀴는듯한 탄닌이 길들여지지 않은야생의 맹수같은 느낌이다. 몇년의 기간을 두고 마시면 깜짝놀랄만한 녀석으로 변할것같다고 조심스레 예상해본다.\n마시자마자 든 느낌은 아주 기름기가 많은 육류와 함께하면 좋을듯하다',
       ' \n진지하면서도 위트가 넘치는 흥미로운 와인. 오레곤을 제외한 미국와인의 특성상 75%이상 사용된 주품종은 레이블에 명시할 수 있는 와인법에 따라 분명 피노누아 이외의 다른 품종이 블렌딩 된 것 같은데 감을 잡을 수 없었다. 찾아보니 글로벌~한 블렌딩. 딸기, 섬세한 체다치즈, 바닐라, 캬라멜 등의 밝은 이미지의 아로마 뒤에 보여지는 우아한 빨레는 이러한 블렌딩의 마법인것 같기도 하다. 우연히 견과류와 함께했는데 크렌베리,캐슈넛과의 시너지 효과가 꽤나 좋았다. 호두는 껍질이 탄닌과 충돌해 에러:)',
       ' \n만들어 먹는 뱅쇼는 그때그때 맛도 다르고 잘못해서 끓기라도 하면 맛이 틀어지는데 크리스트 킨들스는 병째 데워 마시면 온기가 퍼지고 맛 역시 튀는곳 없이 무난해서 좋았다.\n마무리로 올라오는 상큼한 과일맛이 일품 \n전반적으로 산뜻하고 깔끔한 맛~\n너무 강하지도 약하지도 않은 바디감과 산미....매우 괜찮았음\n그냥 데워만 마셔도 좋겠지만 레몬 조각을 살짝 띄워서 마시니 더욱 맛있었는듯..^^',
       ' \n1804 tasting at coex expo \n1804 tasting at coex expo',
       ' \n차갑게 칠링한 로제 스파클링 와인은 특유의 청량감으로 몸과 마음에 신선한 기운을 불어넣어 준다. No.1 호주 와인 브랜드인 제이콥스 크릭이 만든 로제 스파클링은 제철 과일인 딸기나 참외 등과 잘 어울린다. 또, 캠핑에서 즐길 수 있는 조

In [14]:
wine.posting_title.unique()

array(['nan', '   7월  ', '   , 선물하기 좋은  ', '올해 밸런타인데이에 함께할  !',
       '졸업 시즌의 축하주, 스파클링  ', '   , 꼭 한번 마셔볼만한 마트  ',
       '초여름밤, 설레는 기분을 선사해줄  ', "   , 집에서 더 즐겁게 '홈파티  '",
       '늦여름에 마시기 좋은 레드  ', '    - 치명적 우아함, 부르고뉴(Bourgogne)',
       '[ 추천] 소소한 야외 파티에서 즐기기 좋은  ', '   3월  ', '부부의 날을 기념하며 마시기 좋은  ',
       '   2월  ', '호캉스에 단 한 병의  을 가지고 간다면?', '   5월  ', '   6월  ',
       '   8월  ', '   9월  ', '   10월  ', '   11월  ', '   12월  ',
       '   2021년 1월  ', '   4월  ', '폭염이 지속되는 여름밤에 마시고 싶은  ',
       '무더운 날 칠링해서 마시기 좋은 레드  은?', '삼계탕과 잘 어울리는  은?',
       '봄바람과 함께, 야외에서 즐기는  ', '행사가 많은 5월, 선물이 고민인가요?',
       '춘곤증을 날려주는 청량감 넘치는  ', '미세먼지가 심한 날, 홈술 추천  ', '벚꽃 필 무렵엔 이  을!',
       '봄날, 지금 썸타는 중이라면?', " 21's PICK, 카베르네 소비뇽(Cabernet Sauvignon)편",
       '봄을 기다리며 즐기기 좋은  ', '설날 선물을 위한 추천  ', '   , 2021년 ‘소중한  들',
       '이제는 버블을 즐길 시간!', '홈스토랑에서 오픈할  은?',
       '   , 프리미티보 & 진판델(Primitivo & Zinfandel)', '기분전환을 위한 추천  ',
       '찬바람이 불 때는 이  을!', '할로윈을 더 재미있게 만들어줄  ', '차박은 이  과 함께!',
       

In [15]:
wine.head()

,name_ko,type,aroma,maker's_note,tasting_note,posting_title,wine_title,wine_text,hashtag,blog_text
0,더 롱 리틀 독 스파클링,스파클링,"복숭아, 살구, 꽃, 흰꽃, 레몬","밝고 맑은 레몬 컬러에 레몬, 살구, 복숭아, 흰꽃의 아로마가 느껴진다. 입 안에서...",nan,nan,NaN,NaN,NaN,WINE\n더 롱 리틀 독 스파클링 [ The Long Little Dog Spar...
1,루베롱 블랑,NaN,"헤이즐넛, 꽃, 아카시아, 레몬","화사한 노란색을 띠며 레몬, 아카시아 꽃, 헤이즐넛 향이 어우러진다. 입 안에서는 ...",nan,nan,NaN,NaN,NaN,와인이야기\n[하남 포도내음 와인아울렛/와인샵]와인소개_화이트와인추천_Marreno...
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼",스파클링,꽃,"반짝이는 연한 금빛의 정교한 거품을 지녔으며, 흰 꽃 패션 프루츠, 달콤한 흰 과일...",nan,7월,여왕의 샴페인,1867년 랭스(Reims)에 설립된 마리 스튜아트는 스코틀랜드의 여왕이자 프랑스 ...,NaN,BEST와인추천\n와인21닷컴이 선정한 7월 이달의 와인\n와인21 ・ 2021. ...
3,"마스카 델 타코, 로시 피노 네로 로사토",NaN,"딸기, 미네랄",밟은 옅은 분홍색에 야생 딸기 젤리 수박과 같은 달콤한 향이 풍부하고 그 뒤로 향신...,nan,nan,NaN,NaN,NaN,이탈리아와인\nMASCA DEL TACCO [마스카 델 타코] 제이픽와인\n산타 ・...
4,"비에티, 바르베라 다스티 '트레 비그네'",NaN,"체리, 오크",바르베라 다스티의 포도는 라 크레나 포도밭으로 유명한 아스티 언덕주변인 아그리아노 ...,nan,nan,NaN,NaN,NaN,"(N) Piemonte\nVietti, Barbera d'Asti Tre Vigne..."


In [16]:
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 806 entries, 0 to 805
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   name_ko        805 non-null    object
 1   type           68 non-null     object
 2   aroma          321 non-null    object
 3   maker's_note   317 non-null    object
 4   tasting_note   806 non-null    object
 5   posting_title  806 non-null    object
 6   wine_title     290 non-null    object
 7   wine_text      582 non-null    object
 8   hashtag        25 non-null     object
 9   blog_text      301 non-null    object
dtypes: object(10)
memory usage: 63.1+ KB


###### blog_text와 blog_text들을 제외한 모든 열을 합친 새로운 열로 만드는 것이 목표!
###### 이 때, null값이 있으면 합치기 어렵기 때문에 null값에 tokenize 할 때 쉽게 제거할 수 있는 의미 없는 값을 넣음

In [17]:
wine = wine.fillna("ok")

In [18]:
wine.head()

,name_ko,type,aroma,maker's_note,tasting_note,posting_title,wine_title,wine_text,hashtag,blog_text
0,더 롱 리틀 독 스파클링,스파클링,"복숭아, 살구, 꽃, 흰꽃, 레몬","밝고 맑은 레몬 컬러에 레몬, 살구, 복숭아, 흰꽃의 아로마가 느껴진다. 입 안에서...",nan,nan,ok,ok,ok,WINE\n더 롱 리틀 독 스파클링 [ The Long Little Dog Spar...
1,루베롱 블랑,ok,"헤이즐넛, 꽃, 아카시아, 레몬","화사한 노란색을 띠며 레몬, 아카시아 꽃, 헤이즐넛 향이 어우러진다. 입 안에서는 ...",nan,nan,ok,ok,ok,와인이야기\n[하남 포도내음 와인아울렛/와인샵]와인소개_화이트와인추천_Marreno...
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼",스파클링,꽃,"반짝이는 연한 금빛의 정교한 거품을 지녔으며, 흰 꽃 패션 프루츠, 달콤한 흰 과일...",nan,7월,여왕의 샴페인,1867년 랭스(Reims)에 설립된 마리 스튜아트는 스코틀랜드의 여왕이자 프랑스 ...,ok,BEST와인추천\n와인21닷컴이 선정한 7월 이달의 와인\n와인21 ・ 2021. ...
3,"마스카 델 타코, 로시 피노 네로 로사토",ok,"딸기, 미네랄",밟은 옅은 분홍색에 야생 딸기 젤리 수박과 같은 달콤한 향이 풍부하고 그 뒤로 향신...,nan,nan,ok,ok,ok,이탈리아와인\nMASCA DEL TACCO [마스카 델 타코] 제이픽와인\n산타 ・...
4,"비에티, 바르베라 다스티 '트레 비그네'",ok,"체리, 오크",바르베라 다스티의 포도는 라 크레나 포도밭으로 유명한 아스티 언덕주변인 아그리아노 ...,nan,nan,ok,ok,ok,"(N) Piemonte\nVietti, Barbera d'Asti Tre Vigne..."


In [19]:
wine["tasting_note"].isnull().sum()

0

###### 이유는 모르겠으나, fillna로 채워지지 않는 np.nan값은 replace 함수를 써서 대체

In [20]:
wine.replace("nan", "ok", inplace=True)

###### 아래 열들을 합쳐 하나의 열로 만듦

In [21]:
wine["review"] = wine["type"] +"." + wine["aroma"] +"."+ wine["maker's_note"] +"."+ wine["tasting_note"] +"."+ wine["posting_title"] +"."+ wine["wine_title"] +"."+ wine["wine_text"] +"."+ wine["hashtag"] 

In [22]:
wine.head()

,name_ko,type,aroma,maker's_note,tasting_note,posting_title,wine_title,wine_text,hashtag,blog_text,review
0,더 롱 리틀 독 스파클링,스파클링,"복숭아, 살구, 꽃, 흰꽃, 레몬","밝고 맑은 레몬 컬러에 레몬, 살구, 복숭아, 흰꽃의 아로마가 느껴진다. 입 안에서...",ok,ok,ok,ok,ok,WINE\n더 롱 리틀 독 스파클링 [ The Long Little Dog Spar...,"스파클링.복숭아, 살구, 꽃, 흰꽃, 레몬.밝고 맑은 레몬 컬러에 레몬, 살구, 복..."
1,루베롱 블랑,ok,"헤이즐넛, 꽃, 아카시아, 레몬","화사한 노란색을 띠며 레몬, 아카시아 꽃, 헤이즐넛 향이 어우러진다. 입 안에서는 ...",ok,ok,ok,ok,ok,와인이야기\n[하남 포도내음 와인아울렛/와인샵]와인소개_화이트와인추천_Marreno...,"ok.헤이즐넛, 꽃, 아카시아, 레몬.화사한 노란색을 띠며 레몬, 아카시아 꽃, 헤..."
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼",스파클링,꽃,"반짝이는 연한 금빛의 정교한 거품을 지녔으며, 흰 꽃 패션 프루츠, 달콤한 흰 과일...",ok,7월,여왕의 샴페인,1867년 랭스(Reims)에 설립된 마리 스튜아트는 스코틀랜드의 여왕이자 프랑스 ...,ok,BEST와인추천\n와인21닷컴이 선정한 7월 이달의 와인\n와인21 ・ 2021. ...,"스파클링.꽃.반짝이는 연한 금빛의 정교한 거품을 지녔으며, 흰 꽃 패션 프루츠, 달..."
3,"마스카 델 타코, 로시 피노 네로 로사토",ok,"딸기, 미네랄",밟은 옅은 분홍색에 야생 딸기 젤리 수박과 같은 달콤한 향이 풍부하고 그 뒤로 향신...,ok,ok,ok,ok,ok,이탈리아와인\nMASCA DEL TACCO [마스카 델 타코] 제이픽와인\n산타 ・...,"ok.딸기, 미네랄.밟은 옅은 분홍색에 야생 딸기 젤리 수박과 같은 달콤한 향이 풍..."
4,"비에티, 바르베라 다스티 '트레 비그네'",ok,"체리, 오크",바르베라 다스티의 포도는 라 크레나 포도밭으로 유명한 아스티 언덕주변인 아그리아노 ...,ok,ok,ok,ok,ok,"(N) Piemonte\nVietti, Barbera d'Asti Tre Vigne...","ok.체리, 오크.바르베라 다스티의 포도는 라 크레나 포도밭으로 유명한 아스티 언덕..."


###### 필요없어진 열 삭제

In [23]:
del wine["type"]
del wine["aroma"]
del wine["maker's_note"]
del wine["tasting_note"]
del wine["wine_title"]
del wine["wine_text"]
del wine["hashtag"]
del wine["posting_title"]

In [24]:
wine.head()

,name_ko,blog_text,review
0,더 롱 리틀 독 스파클링,WINE\n더 롱 리틀 독 스파클링 [ The Long Little Dog Spar...,"스파클링.복숭아, 살구, 꽃, 흰꽃, 레몬.밝고 맑은 레몬 컬러에 레몬, 살구, 복..."
1,루베롱 블랑,와인이야기\n[하남 포도내음 와인아울렛/와인샵]와인소개_화이트와인추천_Marreno...,"ok.헤이즐넛, 꽃, 아카시아, 레몬.화사한 노란색을 띠며 레몬, 아카시아 꽃, 헤..."
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼",BEST와인추천\n와인21닷컴이 선정한 7월 이달의 와인\n와인21 ・ 2021. ...,"스파클링.꽃.반짝이는 연한 금빛의 정교한 거품을 지녔으며, 흰 꽃 패션 프루츠, 달..."
3,"마스카 델 타코, 로시 피노 네로 로사토",이탈리아와인\nMASCA DEL TACCO [마스카 델 타코] 제이픽와인\n산타 ・...,"ok.딸기, 미네랄.밟은 옅은 분홍색에 야생 딸기 젤리 수박과 같은 달콤한 향이 풍..."
4,"비에티, 바르베라 다스티 '트레 비그네'","(N) Piemonte\nVietti, Barbera d'Asti Tre Vigne...","ok.체리, 오크.바르베라 다스티의 포도는 라 크레나 포도밭으로 유명한 아스티 언덕..."


###### 영어와 기호 의미없는 줄바꿈이 너무 많아 토크나이즈가 제대로 되지 않아, 한글과 숫자 약간의 기호를 제외한 글자들은 모두 삭제함

In [25]:
wine["blog_text"] = wine["blog_text"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9,.\s]","")

In [26]:
wine["review"] = wine["review"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9.,\s]","")

In [27]:
wine.head()

,name_ko,blog_text,review
0,더 롱 리틀 독 스파클링,\n더 롱 리틀 독 스파클링 \n소유리 2021. 6. 29. 1549...,"스파클링.복숭아, 살구, 꽃, 흰꽃, 레몬.밝고 맑은 레몬 컬러에 레몬, 살구, 복..."
1,루베롱 블랑,와인이야기\n하남 포도내음 와인아울렛와인샵와인소개화이트와인추천 마레농 클래식 루...,".헤이즐넛, 꽃, 아카시아, 레몬.화사한 노란색을 띠며 레몬, 아카시아 꽃, 헤이즐..."
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼",와인추천\n와인21닷컴이 선정한 7월 이달의 와인\n와인21 2021. 7. 1....,"스파클링.꽃.반짝이는 연한 금빛의 정교한 거품을 지녔으며, 흰 꽃 패션 프루츠, 달..."
3,"마스카 델 타코, 로시 피노 네로 로사토",이탈리아와인\n 마스카 델 타코 제이픽와인\n산타 2021. 6. 21. 20...,".딸기, 미네랄.밟은 옅은 분홍색에 야생 딸기 젤리 수박과 같은 달콤한 향이 풍부하..."
4,"비에티, 바르베라 다스티 '트레 비그네'","\n, 2018 비에띠, 바르베라 다스티 트레 비녜 2018\n데미안 2...",".체리, 오크.바르베라 다스티의 포도는 라 크레나 포도밭으로 유명한 아스티 언덕주변..."


###### review 열을  토큰화한 뒤 원하는 품사를 가진 단어만 남겨둘 새로운 열 추가

In [28]:
wine["review_token"] = " "

In [29]:
wine = wine[0:325]

###### 원하는 품사(의미를 가진 품사)만 새로운 열에 추가함

In [30]:
for i in range(0,325):
    pos = okt.pos(wine.review[i])
    for j in pos:
        if j[1] == "Adjective":
            wine.review_token[i] = wine.review_token[i] + " " + j[0] 
        if j[1] == "Noun":
            wine.review_token[i] = wine.review_token[i] + " " + j[0] 
        if j[1] == "Number":
            wine.review_token[i] = wine.review_token[i] + " " + j[0]     
        if j[1] == "Suffix":
            wine.review_token[i] = wine.review_token[i] + " " + j[0]
        if j[1] == "Verb":
            wine.review_token[i] = wine.review_token[i] + " " + j[0]

In [31]:
wine.head()

,name_ko,blog_text,review,review_token
0,더 롱 리틀 독 스파클링,\n더 롱 리틀 독 스파클링 \n소유리 2021. 6. 29. 1549...,"스파클링.복숭아, 살구, 꽃, 흰꽃, 레몬.밝고 맑은 레몬 컬러에 레몬, 살구, 복...",스파클링 복숭아 살구 꽃 흰 꽃 레몬 밝고 맑은 레몬 컬러 레몬 살구 복숭아 흰...
1,루베롱 블랑,와인이야기\n하남 포도내음 와인아울렛와인샵와인소개화이트와인추천 마레농 클래식 루...,".헤이즐넛, 꽃, 아카시아, 레몬.화사한 노란색을 띠며 레몬, 아카시아 꽃, 헤이즐...",헤이즐넛 꽃 아카시아 레몬 화사한 노란색 띠 레몬 아카시아 꽃 헤이즐넛 향 어우...
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼",와인추천\n와인21닷컴이 선정한 7월 이달의 와인\n와인21 2021. 7. 1....,"스파클링.꽃.반짝이는 연한 금빛의 정교한 거품을 지녔으며, 흰 꽃 패션 프루츠, 달...",스파클링 꽃 반짝이는 연한 금빛 정교한 거품 지녔으며 흰 꽃 패션 프루츠 달콤한...
3,"마스카 델 타코, 로시 피노 네로 로사토",이탈리아와인\n 마스카 델 타코 제이픽와인\n산타 2021. 6. 21. 20...,".딸기, 미네랄.밟은 옅은 분홍색에 야생 딸기 젤리 수박과 같은 달콤한 향이 풍부하...",딸기 미네랄 밟은 옅은 분홍색 야생 딸기 젤리 수박 같은 달콤한 향 풍부하고 그...
4,"비에티, 바르베라 다스티 '트레 비그네'","\n, 2018 비에띠, 바르베라 다스티 트레 비녜 2018\n데미안 2...",".체리, 오크.바르베라 다스티의 포도는 라 크레나 포도밭으로 유명한 아스티 언덕주변...",체리 오크 바르 베 다스 티 포도 크 레나 포도 밭 유명한 아스티 언덕 주변인 ...


In [32]:
for i in range(0,325):
    wine["review_token"][i] = wine["review_token"][i].strip()

###### 빈도수순으로 단어들이 새롭게 정렬될 열을 만듭니다.

In [33]:
wine["sorted_review_token"] = " "

###### 빈값이 있으면 아래 반복문이 돌아가지 않기 때문에 의미없는 글자로 채워줍니다.

In [34]:
wine['review_token'].replace('', "이", inplace=True)

###### 와인 당 개별로 빈도수를 측정해서  Dictionary를 만들고,  중복을 제거하고, 빈도수순으로 새롭게 정렬함.

In [35]:
for n in range(0,325):
    
    vocab ={}
    word_to_index ={}    
    word_to_index_reverse = {}
    sentence=[]
    sentences=[]   
    i = 0
    
    for word in word_tokenize(wine.review_token[n]):
         if word not in vocab:
            vocab[word] = 0
        vocab[word] +=1
        
         vocab_sorted = sorted(vocab.items(), key=lambda x: x[1], reverse=True)
    
    for word,frequency in vocab_sorted:
         i+=1
        word_to_index[word]=i

    
    for word in word_tokenize(wine.review_token[n]):
         if word in word_to_index.keys():
                    if word_to_index[word] not in sentence:
                          sentence.append(word_to_index[word])
        sentence = sorted(sentence)
         wine["sorted_review_token"][n] = sentence   
       
        
    word_to_index_reverse = {v:k for k,v in word_to_index.items()}

    for number in wine.sorted_review_token[n]:
         sentences.append(word_to_index_reverse[number])
        wine["review_token"][n] = sentences
    

In [36]:
wine.head()

,name_ko,blog_text,review,review_token,sorted_review_token
0,더 롱 리틀 독 스파클링,\n더 롱 리틀 독 스파클링 \n소유리 2021. 6. 29. 1549...,"스파클링.복숭아, 살구, 꽃, 흰꽃, 레몬.밝고 맑은 레몬 컬러에 레몬, 살구, 복...","[꽃, 레몬, 복숭아, 살구, 흰, 아로마, 스파클링, 밝고, 맑은, 컬러, 느껴진...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,루베롱 블랑,와인이야기\n하남 포도내음 와인아울렛와인샵와인소개화이트와인추천 마레농 클래식 루...,".헤이즐넛, 꽃, 아카시아, 레몬.화사한 노란색을 띠며 레몬, 아카시아 꽃, 헤이즐...","[헤이즐넛, 꽃, 아카시아, 레몬, 화사한, 노란색, 띠, 향, 어우러진다, 입, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼",와인추천\n와인21닷컴이 선정한 7월 이달의 와인\n와인21 2021. 7. 1....,"스파클링.꽃.반짝이는 연한 금빛의 정교한 거품을 지녔으며, 흰 꽃 패션 프루츠, 달...","[샴페인, 피노, 마리, 스튜, 아트, 뀌베, 여왕, 드, 꽃, 정교한, 흰, 있다...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,"마스카 델 타코, 로시 피노 네로 로사토",이탈리아와인\n 마스카 델 타코 제이픽와인\n산타 2021. 6. 21. 20...,".딸기, 미네랄.밟은 옅은 분홍색에 야생 딸기 젤리 수박과 같은 달콤한 향이 풍부하...","[향, 딸기, 미네랄, 밟은, 옅은, 분홍색, 야생, 젤리, 수박, 같은, 달콤한,...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,"비에티, 바르베라 다스티 '트레 비그네'","\n, 2018 비에띠, 바르베라 다스티 트레 비녜 2018\n데미안 2...",".체리, 오크.바르베라 다스티의 포도는 라 크레나 포도밭으로 유명한 아스티 언덕주변...","[체리, 오크, 포도, 향, 느껴진다, 바르, 베, 다스, 티, 크, 레나, 밭, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


###### 필요 없어진 열 삭제

In [37]:
del wine["review"]
del wine["sorted_review_token"]

In [38]:
wine.head()

,name_ko,blog_text,review_token
0,더 롱 리틀 독 스파클링,\n더 롱 리틀 독 스파클링 \n소유리 2021. 6. 29. 1549...,"[꽃, 레몬, 복숭아, 살구, 흰, 아로마, 스파클링, 밝고, 맑은, 컬러, 느껴진..."
1,루베롱 블랑,와인이야기\n하남 포도내음 와인아울렛와인샵와인소개화이트와인추천 마레농 클래식 루...,"[헤이즐넛, 꽃, 아카시아, 레몬, 화사한, 노란색, 띠, 향, 어우러진다, 입, ..."
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼",와인추천\n와인21닷컴이 선정한 7월 이달의 와인\n와인21 2021. 7. 1....,"[샴페인, 피노, 마리, 스튜, 아트, 뀌베, 여왕, 드, 꽃, 정교한, 흰, 있다..."
3,"마스카 델 타코, 로시 피노 네로 로사토",이탈리아와인\n 마스카 델 타코 제이픽와인\n산타 2021. 6. 21. 20...,"[향, 딸기, 미네랄, 밟은, 옅은, 분홍색, 야생, 젤리, 수박, 같은, 달콤한,..."
4,"비에티, 바르베라 다스티 '트레 비그네'","\n, 2018 비에띠, 바르베라 다스티 트레 비녜 2018\n데미안 2...","[체리, 오크, 포도, 향, 느껴진다, 바르, 베, 다스, 티, 크, 레나, 밭, ..."


###### 위와 마찬가지로 blog_text 열을  토큰화한 뒤 원하는 품사를 가진 단어만 남겨둘 새로운 열 추가

In [39]:
wine["blog_token"] = " "

###### 원하는 품사(의미를 가진 품사)만 새로운 열에 추가함

In [40]:
pos_okt = okt.pos(wine.blog_text[1], norm=True, stem=True)

In [41]:
for i in range(0,325):
    pos = okt.pos(wine.blog_text[i], norm=True, stem=True)
    for j in pos:
        if j[1] == "Verb":
            wine.blog_token[i] = wine.blog_token[i] + " " + j[0] 
        if j[1] == "Noun":
            wine.blog_token[i] = wine.blog_token[i] + " " + j[0] 
        if j[1] == "Adjective":
            wine.blog_token[i] = wine.blog_token[i] + " " + j[0] 

###### 단어의 빈도수를 확인하고, 블로그의 특성상 존재하는 관련없는 말, 의미없는 말을 제거하기 위해 빈도수가 3이 넘는 값만 중복을 제거하여 남겨둠

In [42]:
wine.head()

,name_ko,blog_text,review_token,blog_token
0,더 롱 리틀 독 스파클링,\n더 롱 리틀 독 스파클링 \n소유리 2021. 6. 29. 1549...,"[꽃, 레몬, 복숭아, 살구, 흰, 아로마, 스파클링, 밝고, 맑은, 컬러, 느껴진...",더 롱 리틀 독 스파클링 소유리 복사 이웃 추가 더 롱 리틀 독 스파클링 단맛 ...
1,루베롱 블랑,와인이야기\n하남 포도내음 와인아울렛와인샵와인소개화이트와인추천 마레농 클래식 루...,"[헤이즐넛, 꽃, 아카시아, 레몬, 화사한, 노란색, 띠, 향, 어우러진다, 입, ...",와인 이야기 하남 포도 음 와인 아울렛 와인 와인 소개 화이트와인 추천 마레 농...
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼",와인추천\n와인21닷컴이 선정한 7월 이달의 와인\n와인21 2021. 7. 1....,"[샴페인, 피노, 마리, 스튜, 아트, 뀌베, 여왕, 드, 꽃, 정교한, 흰, 있다...",와인 추천 와인 컴 선정 달 와인 와인 복사 이웃 추가 조금 늦다 찾아오다 장마...
3,"마스카 델 타코, 로시 피노 네로 로사토",이탈리아와인\n 마스카 델 타코 제이픽와인\n산타 2021. 6. 21. 20...,"[향, 딸기, 미네랄, 밟은, 옅은, 분홍색, 야생, 젤리, 수박, 같은, 달콤한,...",이탈리아 와인 마 스카 델 타코 제이 픽 와인 산타 복사 이웃 추가 마 스카 델...
4,"비에티, 바르베라 다스티 '트레 비그네'","\n, 2018 비에띠, 바르베라 다스티 트레 비녜 2018\n데미안 2...","[체리, 오크, 포도, 향, 느껴진다, 바르, 베, 다스, 티, 크, 레나, 밭, ...",비 띠 바르다 베 다스 티 트레 비녜 데미안 복사 이웃 추가 비 띠 바르다 베 ...


In [43]:
wine_copy=wine.copy()

In [44]:
for i in range(0,325):
  wine_copy["blog_token"][i] = wine_copy["blog_token"][i].strip()

###### 정렬된 인코딩된 단어가 들어갈  새로운 열을 만듦

In [45]:
wine_copy["sorted_blog_token"] = " "

###### 비어있는 값을 의미없는 말로 대체

In [46]:
wine_copy['blog_token'].replace('', "이", inplace=True)

###### review와 마찬가지로 와인 당 개별로 빈도수를 측정해서 Dictionary를 만들고, 중복을 제거하고, 빈도수순으로 새롭게 정렬함

In [47]:
for n in range(0,325):
    
    vocab ={}
    word_to_index ={}    
    word_to_index_reverse = {}
    sentence=[]
    sentences=[]   
    i = 0
    
    for word in word_tokenize(wine_copy.blog_token[n]):
         if word not in vocab:
             vocab[word] = 0
         vocab[word] +=1
        
         vocab_sorted = sorted(vocab.items(), key=lambda x: x[1], reverse=True)
    
    for word,frequency in vocab_sorted:
         i+=1
         word_to_index[word]=i

    
    for word in word_tokenize(wine_copy.blog_token[n]):
         if word in word_to_index.keys():
             if word_to_index[word] not in sentence:
                  sentence.append(word_to_index[word])
         sentence = sorted(sentence)
         wine_copy["sorted_blog_token"][n] = sentence   
       
        
    word_to_index_reverse = {v:k for k,v in word_to_index.items()}

    for number in wine_copy.sorted_blog_token[n]:
         sentences.append(word_to_index_reverse[number])
         wine_copy["blog_token"][n] = sentences


In [48]:
wine_copy.head()

,name_ko,blog_text,review_token,blog_token,sorted_blog_token
0,더 롱 리틀 독 스파클링,\n더 롱 리틀 독 스파클링 \n소유리 2021. 6. 29. 1549...,"[꽃, 레몬, 복숭아, 살구, 흰, 아로마, 스파클링, 밝고, 맑은, 컬러, 느껴진...","[와인, 하다, 있다, 맛, 것, 더, 독, 같다, 좋다, 느껴지다, 롱, 리틀, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,루베롱 블랑,와인이야기\n하남 포도내음 와인아울렛와인샵와인소개화이트와인추천 마레농 클래식 루...,"[헤이즐넛, 꽃, 아카시아, 레몬, 화사한, 노란색, 띠, 향, 어우러진다, 입, ...","[와인, 하다, 있다, 루베, 롱, 블, 포도, 지역, 품종, 이, 이다, 프랑스,...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼",와인추천\n와인21닷컴이 선정한 7월 이달의 와인\n와인21 2021. 7. 1....,"[샴페인, 피노, 마리, 스튜, 아트, 뀌베, 여왕, 드, 꽃, 정교한, 흰, 있다...","[하다, 오페라, 있다, 되다, 의, 그, 되어다, 와인, 늘다, 오다, 음악, 작...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,"마스카 델 타코, 로시 피노 네로 로사토",이탈리아와인\n 마스카 델 타코 제이픽와인\n산타 2021. 6. 21. 20...,"[향, 딸기, 미네랄, 밟은, 옅은, 분홍색, 야생, 젤리, 수박, 같은, 달콤한,...","[와인, 하다, 마, 스카, 점, 있다, 델, 타코, 품종, 향, 피다, 이탈리아,...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,"비에티, 바르베라 다스티 '트레 비그네'","\n, 2018 비에띠, 바르베라 다스티 트레 비녜 2018\n데미안 2...","[체리, 오크, 포도, 향, 느껴진다, 바르, 베, 다스, 티, 크, 레나, 밭, ...","[하다, 바르다, 베, 티, 와인, 비, 다스, 있다, 띠, 이다, 말다, 감, 자...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


###### 혹시모를일에 대비해 복사본을 만들어둠

In [49]:
wine = wine_copy.copy()

###### 필요없어진열 삭제

In [50]:
del wine["blog_text"]
del wine["sorted_blog_token"]

In [51]:
wine.head()

,name_ko,review_token,blog_token
0,더 롱 리틀 독 스파클링,"[꽃, 레몬, 복숭아, 살구, 흰, 아로마, 스파클링, 밝고, 맑은, 컬러, 느껴진...","[와인, 하다, 있다, 맛, 것, 더, 독, 같다, 좋다, 느껴지다, 롱, 리틀, ..."
1,루베롱 블랑,"[헤이즐넛, 꽃, 아카시아, 레몬, 화사한, 노란색, 띠, 향, 어우러진다, 입, ...","[와인, 하다, 있다, 루베, 롱, 블, 포도, 지역, 품종, 이, 이다, 프랑스,..."
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼","[샴페인, 피노, 마리, 스튜, 아트, 뀌베, 여왕, 드, 꽃, 정교한, 흰, 있다...","[하다, 오페라, 있다, 되다, 의, 그, 되어다, 와인, 늘다, 오다, 음악, 작..."
3,"마스카 델 타코, 로시 피노 네로 로사토","[향, 딸기, 미네랄, 밟은, 옅은, 분홍색, 야생, 젤리, 수박, 같은, 달콤한,...","[와인, 하다, 마, 스카, 점, 있다, 델, 타코, 품종, 향, 피다, 이탈리아,..."
4,"비에티, 바르베라 다스티 '트레 비그네'","[체리, 오크, 포도, 향, 느껴진다, 바르, 베, 다스, 티, 크, 레나, 밭, ...","[하다, 바르다, 베, 티, 와인, 비, 다스, 있다, 띠, 이다, 말다, 감, 자..."


In [52]:
from nltk.corpus import stopwords

###### 자주 등장하지만 의미없는 말을 stopword를 사용하여 약간 걸러냄

In [53]:
review_stop_word = ["와인", "적", "있다", "잘", "수", "이", "있는", "띠", "된", "등", "하는", "한다", "해", "들", "안", "너리", "피노", "로", "의", "하며","미가", "후", "같은", "밭", "할", "되는", "된다", "중", "월간", "이상", "를", "약간", "동안", "매우", "했다", "되었다", "통해", "뒤", "및", "있고", "입니다", "듯", "오슈", "되고", "자", "만", "페", "멘", "까"]

In [54]:
blog_stop_word =["와인","복사", "이웃", "추가", "하다", "있다", "되다", "이", "품종", "너리", "이다", "수", "것", "점", "지역", "그", "보다", "않다", "자다", "점수", "비비다", "피다", "등", "맛", "같다", "만들다", "더", "로", "누", "소비", "샴페인", "되어다", "의", "오다", "뇽", "바", "도", "입사", "밉다", "가다", "가격", "드", "느낌", "느껴지다", "국가", "중" , "없다", "비", "받다", "리", "블", "마시다", "가장", "추가", "정도", "말다", "늘다", "끄다","사용", "이웃", "메다", "밭", "아니다", "때", "판매", "복사", "들다", "데", "레", "먹다", "밸리", "병", "곳", "위", "높다", "를", "많다",
"이름", "감", "후", "재배", "샤",  "생각", "처", "음", "시작", "날" , "개", "입", "트", "시라", "다른", "시간", "뒤", "전", "테이", "그렇다", "및","스", "티", "지다", "메", "싶다", "이상", "약간", "내다", "듯", "코", "쉬라즈", "하나", "특징", "주다", "때문", "스팅", "디", "좀", "아주", "나", "바로", "위해", "알다", "테이블", "안", "빈야드", "제", "내", "풀", "나다", "자리", "슬링", "함","론", "알", "브", "또", "은", "소개", "보이다", "링", "색", "이렇다",  "베", "두", "번", "제르", "이후", "포", "조금", "테", "시", "해주다", "일", "르", "통", "종", "정말", "저"]

###### 결과값이 들어갈 새로운 행을 만듦

In [55]:
wine["blogreview"] = " "

###### list를 string으로 변환하기 위한 새로운 열을 만듦

In [56]:
wine["review"] = " "

In [58]:
wine["blog"] = " "

###### list를 string으로 변환함

In [57]:
for i in range(0,325):
    wine["review"][i] = " ".join(wine['review_token'][i])

In [59]:
for i in range(0,325):
    wine["blog"][i] = " ".join(wine['blog_token'][i])

###### review_text와 blog_text를 합치기 전에 csv파일을 만들어둠

In [60]:
wine.to_csv("wine_tokenizing_review_blog")

###### stopword를 제거하고, 좀 더 전문성을 가진 review 데이터에 우선순위를 둬서 중복없이 두개의 열을 합침

In [61]:
for i in range(0,325):  
    sentence=[]
    for word in word_tokenize(wine.review[i]):
         if word not in review_stop_word:
                if word not in sentence:
                    sentence.append(word)
    for word in word_tokenize(wine.blog[i]):
         if word not in blog_stop_word:
                if word not in sentence:
                    sentence.append(word)

    wine["blogreview"][i] = sentence

In [62]:
wine.head()

,name_ko,review_token,blog_token,blogreview,review,blog
0,더 롱 리틀 독 스파클링,"[꽃, 레몬, 복숭아, 살구, 흰, 아로마, 스파클링, 밝고, 맑은, 컬러, 느껴진...","[와인, 하다, 있다, 맛, 것, 더, 독, 같다, 좋다, 느껴지다, 롱, 리틀, ...","[꽃, 레몬, 복숭아, 살구, 흰, 아로마, 스파클링, 밝고, 맑은, 컬러, 느껴진...",꽃 레몬 복숭아 살구 흰 아로마 스파클링 밝고 맑은 컬러 느껴진다 입 안 싱그러운 ...,와인 하다 있다 맛 것 더 독 같다 좋다 느껴지다 롱 리틀 가볍다 음식 먹다 말다 ...
1,루베롱 블랑,"[헤이즐넛, 꽃, 아카시아, 레몬, 화사한, 노란색, 띠, 향, 어우러진다, 입, ...","[와인, 하다, 있다, 루베, 롱, 블, 포도, 지역, 품종, 이, 이다, 프랑스,...","[헤이즐넛, 꽃, 아카시아, 레몬, 화사한, 노란색, 향, 어우러진다, 입, 부드럽...",헤이즐넛 꽃 아카시아 레몬 화사한 노란색 띠 향 어우러진다 입 안 부드럽고 섬세한 ...,와인 하다 있다 루베 롱 블 포도 지역 품종 이 이다 프랑스 되다 추천 자다 생산 ...
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼","[샴페인, 피노, 마리, 스튜, 아트, 뀌베, 여왕, 드, 꽃, 정교한, 흰, 있다...","[하다, 오페라, 있다, 되다, 의, 그, 되어다, 와인, 늘다, 오다, 음악, 작...","[샴페인, 마리, 스튜, 아트, 뀌베, 여왕, 드, 꽃, 정교한, 흰, 우아하고, ...",샴페인 피노 마리 스튜 아트 뀌베 여왕 드 꽃 정교한 흰 있다 우아하고 균형 감 생...,하다 오페라 있다 되다 의 그 되어다 와인 늘다 오다 음악 작품 세기 것 이 를 수...
3,"마스카 델 타코, 로시 피노 네로 로사토","[향, 딸기, 미네랄, 밟은, 옅은, 분홍색, 야생, 젤리, 수박, 같은, 달콤한,...","[와인, 하다, 마, 스카, 점, 있다, 델, 타코, 품종, 향, 피다, 이탈리아,...","[향, 딸기, 미네랄, 밟은, 옅은, 분홍색, 야생, 젤리, 수박, 달콤한, 풍부하...",향 딸기 미네랄 밟은 옅은 분홍색 야생 젤리 수박 같은 달콤한 풍부하고 그 뒤 향신...,와인 하다 마 스카 점 있다 델 타코 품종 향 피다 이탈리아 생산 신선하다 지역 이...
4,"비에티, 바르베라 다스티 '트레 비그네'","[체리, 오크, 포도, 향, 느껴진다, 바르, 베, 다스, 티, 크, 레나, 밭, ...","[하다, 바르다, 베, 티, 와인, 비, 다스, 있다, 띠, 이다, 말다, 감, 자...","[체리, 오크, 포도, 향, 느껴진다, 바르, 베, 다스, 티, 크, 레나, 유명한...",체리 오크 포도 향 느껴진다 바르 베 다스 티 크 레나 밭 유명한 아스티 언덕 주변...,하다 바르다 베 티 와인 비 다스 있다 띠 이다 말다 감 자다 오크 품종 것 추가 ...


###### 필요없어진열 삭제

In [63]:
del wine["review_token"]
del wine["blog_token"]
del wine["review"]
del wine["blog"]

In [64]:
wine.head()

,name_ko,blogreview
0,더 롱 리틀 독 스파클링,"[꽃, 레몬, 복숭아, 살구, 흰, 아로마, 스파클링, 밝고, 맑은, 컬러, 느껴진..."
1,루베롱 블랑,"[헤이즐넛, 꽃, 아카시아, 레몬, 화사한, 노란색, 향, 어우러진다, 입, 부드럽..."
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼","[샴페인, 마리, 스튜, 아트, 뀌베, 여왕, 드, 꽃, 정교한, 흰, 우아하고, ..."
3,"마스카 델 타코, 로시 피노 네로 로사토","[향, 딸기, 미네랄, 밟은, 옅은, 분홍색, 야생, 젤리, 수박, 달콤한, 풍부하..."
4,"비에티, 바르베라 다스티 '트레 비그네'","[체리, 오크, 포도, 향, 느껴진다, 바르, 베, 다스, 티, 크, 레나, 유명한..."


###### 결과값을 다음에 더 쉽게 변환할 수 있도록 list에서 string으로 변경

In [65]:
for i in range(0,325):
    wine["blogreview"][i] = " ".join(wine['blogreview'][i])

In [66]:
wine.head()

,name_ko,blogreview
0,더 롱 리틀 독 스파클링,꽃 레몬 복숭아 살구 흰 아로마 스파클링 밝고 맑은 컬러 느껴진다 입 싱그러운 상큼...
1,루베롱 블랑,헤이즐넛 꽃 아카시아 레몬 화사한 노란색 향 어우러진다 입 부드럽고 섬세한 맛 생기...
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼",샴페인 마리 스튜 아트 뀌베 여왕 드 꽃 정교한 흰 우아하고 균형 감 생산 누 샤르...
3,"마스카 델 타코, 로시 피노 네로 로사토",향 딸기 미네랄 밟은 옅은 분홍색 야생 젤리 수박 달콤한 풍부하고 그 향신료 광물 ...
4,"비에티, 바르베라 다스티 '트레 비그네'",체리 오크 포도 향 느껴진다 바르 베 다스 티 크 레나 유명한 아스티 언덕 주변인 ...


###### 결과값을 csv파일로 저장함

In [67]:
wine.to_csv("wine_tokenizing_final")